In [36]:
import re
import time
import requests
import selenium
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go

from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from sympy import symbols, solve
from plotly.subplots import make_subplots
from pandas_datareader import data as pdr
from selenium.webdriver.common.by import By
from forex_python.converter import CurrencyRates
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService

In [37]:
ticker = str(input('Ticker: '))

In [38]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

url = fr'https://finance.yahoo.com/quote/{ticker}?p={ticker}&.tsrc=fin-srch'

driver.get(url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

market_cap_parent = soup.find('span', string = 'Market Cap').find_parent('tr')
market_cap_element = market_cap_parent.find('td', class_ = "Ta(end) Fw(600) Lh(14px)").text

value, unit = market_cap_element[:-1], market_cap_element[-1]
value = float(value)

if unit == 'T':
    market_cap_value = value * 1e12
elif unit == 'B':
    market_cap_value = value * 1e9
elif unit == 'M':
    market_cap_value = value * 1e6

print(f'Market cap value: {round(market_cap_value):,}')


beta_parent = soup.find('span', string = 'Beta (5Y Monthly)').find_parent('tr')
beta_value = float(beta_parent.find('td', class_ = "Ta(end) Fw(600) Lh(14px)").text)
print(f'Beta: {beta_value:,}')

driver.quit()

Market cap value: 107,516,000,000
Beta: 0.99


In [39]:
options = Options()

options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

financial_url = fr'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
driver.get(financial_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

pretax_income_parent = soup.find('div', class_ = "D(ib) Va(m) Ell Mt(-3px) W(215px)--mv2 W(200px) undefined", title = "Pretax Income")
pretax_income_span = pretax_income_parent.find_next('span').find_next('span')
pretax_income_value = int(re.sub(',', '', pretax_income_span.text)) * 1000
print(f"Pretax Income: {pretax_income_value:,}")

tax_provision_parent = soup.find('div',  class_ = "D(ib) Va(m) Ell Mt(-3px) W(215px)--mv2 W(200px) undefined", title = "Tax Provision")
tax_provision_span = tax_provision_parent.find_next('span').find_next('span')
tax_provision_value = int(re.sub(',', '', tax_provision_span.text)) * 1000
print(f"Tax Provision: {tax_provision_value:,}")

interest_expense_parent = soup.find('div',  class_ = "D(ib) Va(m) Ell Mt(-3px) W(215px)--mv2 W(200px) undefined", title = "Interest Expense")
interest_expense_span = interest_expense_parent.find_next('span').find_next('span')
interest_expense_value = int(re.sub(',', '', interest_expense_span.text)) * 1000
print(f"Interest Expense: {interest_expense_value:,}")

driver.quit()

Pretax Income: 5,646,000,000
Tax Provision: 1,352,100,000
Interest Expense: 560,500,000


In [40]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

balance_sheet_url = fr'https://stockanalysis.com/stocks/{ticker}/financials/balance-sheet/'
driver.get(balance_sheet_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

debt_td = soup.find('span', string = 'Total Debt').find_parent('td').find_next('td').text
debt_value = float(re.sub(',', '', debt_td))* 1000000
print(f'Total Debt: {debt_value:,}')

driver.quit()

Total Debt: 24,600,000,000.0


In [41]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

treasury_bond_url = r'https://ycharts.com/indicators/10_year_treasury_rate'
driver.get(treasury_bond_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

tnx_row = soup.find('td', string = 'Last Value').find_next('td').text
tnx_value = float(re.sub('%', '', tnx_row))

driver.quit()

In [42]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

financial_url = r'https://finance.yahoo.com/quote/VTI/performance?p=VTI'
driver.get(financial_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

return_div = soup.find('span', string = '10-Year').find_parent('div')
vti_10_year_return = return_div.find_all('span', class_ = 'W(20%)')[0].text.strip()
market_value = float(re.sub('%', '', vti_10_year_return))
print(f'Expected Market Return (10-year VTI): {market_value} %')
print(f'The value of 10-year Treasury Bond Yield is: {tnx_value} %')
print(f'Market Premium: {round(market_value - tnx_value, 4)} %')

driver.quit()

Expected Market Return (10-year VTI): 11.91 %
The value of 10-year Treasury Bond Yield is: 4.27 %
Market Premium: 7.64 %


$WACC\ (Weighted\ Average\ Cost\ of\ Capital) = W_D * R_D * (1-t) + W_E * R_E$ <br><br>
Weight of Debt ($W_D$) = $\frac{Debt}{Debt\ +\ Market\ Cap}$ <br><br>
Weight of Equity ($W_E$) = $\frac{Market\ Cap}{Debt\ +\ Market\ Cap}$ <br><br>
Cost of Debt ($R_D$) = $\frac{Interest\ Expense}{Debt}$ <br><br>
Cost of Equity ($R_E$) = $Treasury\ Bond\ Rate\ +\ Beta * (Expected\ Market\ Return - Treasury\ Bond\ Rate\ )$

In [43]:
weight_of_debt = debt_value / (debt_value + market_cap_value)
weight_of_equity = market_cap_value / (debt_value + market_cap_value)
cost_of_debt = interest_expense_value/ debt_value
cost_of_equity = (tnx_value + beta_value * (market_value - tnx_value)) / 100
tax_rate = tax_provision_value / pretax_income_value
wacc = (weight_of_debt * cost_of_debt * (1 - tax_rate) + weight_of_equity * cost_of_equity) * 100

print(f'Weight of Debt = {round(weight_of_debt * 100, 4)} %')
print(f'Weight of Equity = {round(weight_of_equity * 100, 4)} %')
print(f'Cost of Debt = {round(cost_of_debt * 100, 4)} %')
print(f'Cost of Euqity = {round(cost_of_equity * 100, 4)} %')
print('========================')
print(f'WACC = {round(wacc, 4)} %')

Weight of Debt = 18.62 %
Weight of Equity = 81.38 %
Cost of Debt = 2.2785 %
Cost of Euqity = 11.8336 %
WACC = 9.9528 %


In [44]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

fcf_url = fr'https://stockanalysis.com/stocks/{ticker}/financials/'
driver.get(fcf_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

fcf_td = soup.find('span', string = 'Free Cash Flow').find_parent('td').find_next('td').text
fcf_value = float(re.sub(',', '', fcf_td)) * 1000000

driver.quit()

In [45]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

sa_url = fr'https://stockanalysis.com/stocks/{ticker}/forecast/'
driver.get(sa_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

revenue_growth_numbers = []

revenue_growth_td = soup.find('tr', class_ = "border-b text-[0.85rem] dark:border-dark-700 sm:text-[0.95rem]").find_all('td')

for td in range(6, 11):
    revenue_growth_numbers.append(float(re.sub('%', '', revenue_growth_td[td]['title'])))

growth_rate_1 = (revenue_growth_numbers[0] / 100) + 1
growth_rate_2 = (revenue_growth_numbers[1] / 100) + 1
growth_rate_3 = (revenue_growth_numbers[2] / 100) + 1
growth_rate_4 = (revenue_growth_numbers[3] / 100) + 1
growth_rate_5 = (revenue_growth_numbers[4] / 100) + 1
terminal_growth_rate = 1.025
print(f"Growth Rate of Further 5 years and Terminal Growth Rate: {revenue_growth_numbers[0]}%, {revenue_growth_numbers[1]}%, {revenue_growth_numbers[2]}%, {revenue_growth_numbers[3]}%, {revenue_growth_numbers[4]}%, {round((terminal_growth_rate - 1) * 100, 3)}%")

driver.quit()

Growth Rate of Further 5 years and Terminal Growth Rate: 11.684%, 9.965%, 9.594%, 12.178%, 8.905%, 2.5%


### Terminal Value
[Wall Street Oasis | Terminal Value](https://www.wallstreetoasis.com/resources/skills/valuation/terminal-value)  
[Wall Street Oasis | EBIT vs EBITDA](https://www.wallstreetoasis.com/resources/skills/finance/exit-multiple)  

#### Exit Multiple Method
The exit multiple method assumes that a company will **be sold after the forecast period** for a multiple of some market indicator.  
$$
Terminal\ Value\ = Final\ Year\ Metric\ *\ Exit\ Multiple
$$

Advantages:  
- Reflecting the market expectations and valuation multiples of comparable companies.
- Using market-based data rather than subjective assumptions about growth rates and margins.

Drawbacks:
- It is more difficult and subjective to find and select appropriate multiples and comparable companies.
- It is more uncertain and volatile, depending on market conditions and sentiments that may change over time or vary across different sectors.

##### EBITDA multiple
- **Mature & Stable** companies with consistent cash flows and margins
- It may NOT be suitable for **high-growth** or **low-margin** companies like technology or biotech.

##### EBIT multiple
- Include depreciation and amortization, which is particularly useful while analyzing **capital-intensive** businesses where depreciation is a true economic cost.
- Such as, automobile manufacturing, energy, steel production, and telecommunications.

##### Revenue multiple
- Often used for **high-growth** or **low-margin** companies with strong revenue potential but **not profitable** or have **negative cash flows**.
- It may NOT be suitable for companies with **stable revenue but declining growth**.

##### Earnings multiple
- Often used for profitable and growing companies with **positive cash flows and earnings**, such as **consumer discretionary**.

---

#### Perpetual Growth Method
The perpetual growth method assumes that a company will **always produce cash flows at a steady rate** after the projection time.  
$$
Terminal\ Value = \frac{Final\ Year\ FCF\ *\ (1\ +\ Perpetuity\ Growth\ Rate)}{(Discount\ Rate\ -\ Perpetuity\ Growth\ Rate)}
$$

Advantages:
- Consist with the theory of discounted cash flow valuation.
- It is widely used and accepted by academics and practitioners.

Drawbacks:
- It is sensitive to the assumptions of the growth rate and the discount rate.
- Improbable in a dynamic and competitive market.
- This is because it ignores the potential changes in the industry structure.

In [47]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

share_url = fr'https://stockanalysis.com/stocks/{ticker}/statistics/'
driver.get(share_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

## EV / EBIT multiple
ebit_td = soup.find('span', string = 'EBIT ').find_parent('td').find_next('td')['title']
ebit_num = float(re.sub(',', '', ebit_td))

evebit_td = soup.find('span', string = 'EV / EBIT ').find_parent('td').find_next('td').text
evebit_num = float(evebit_td)

## EV / EBITDA multiple
ebitda_td = soup.find('span', string = 'EBITDA ').find_parent('td').find_next('td')['title']
ebitda_num = float(re.sub(',', '', ebitda_td))

evebitda_td = soup.find('span', string = 'EV / EBITDA ').find_parent('td').find_next('td').text
evebitda_num = float(evebitda_td)

# print(f"EBITDA: {ebitda_num:,}")
# print(f"EV / EBITDA: {evebitda_num:,}")
# print(f"Terminal Value by EBITDA multiple: {ebitda_num * evebitda_num:,}")

## Share Outstandings
share_td = soup.find('span', string = 'Shares Outstanding ').find_parent('td').find_next('td')['title']
share_num = float(re.sub(',', '', share_td))
# print(f"Shares Outstanding: {share_num:,}")

## Current Stock Price
## During 休盤
# price = float(soup.find('div', class_ = "text-4xl font-bold block sm:inline").text)

## During 開盤
price = float(soup.find('div', class_ = "text-4xl font-bold inline-block").text)

driver.quit()

In [48]:
current_datetime = datetime.now()
current_year = current_datetime.year
end_of_year = datetime(current_datetime.year, 12, 31)
time_difference = end_of_year - current_datetime
percentage_remaining = (time_difference.days / 365)

fcf_ttm = round(fcf_value)
fcf_1 = round(fcf_ttm * growth_rate_1)
dcf_1 = round(fcf_1 / ((1 + (wacc / 100)) ** (percentage_remaining / 2)))  ## DCF in the middle of this year whatever the remaining

fcf_2 = round(fcf_1 * growth_rate_2)
dcf_2 = round(fcf_2 / ((1 + (wacc / 100)) ** (percentage_remaining + 0.5))) ## The end of this year + 0.5 year = Middle of the next year

fcf_3 = round(fcf_2 * growth_rate_3)
dcf_3 = round(fcf_3 / ((1 + (wacc / 100)) ** (percentage_remaining + 1.5)))

fcf_4 = round(fcf_3 * growth_rate_4)
dcf_4 = round(fcf_4 / ((1 + (wacc / 100)) ** (percentage_remaining + 2.5)))

fcf_5 = round(fcf_4 * growth_rate_5)
dcf_5 = round(fcf_5 / ((1 + (wacc / 100)) ** (percentage_remaining + 3.5)))

## Perpetual Growth Method
# dcf_terminal = round((fcf_5 * terminal_growth_rate / ((wacc / 100) - (terminal_growth_rate - 1))) / ((1 + (wacc / 100)) ** (percentage_remaining + 3.5)))

## Exit Multiple Method EV/EBIT, Capital-Intensive Industries
# dcf_terminal = round((ebit_num * evebit_num) / ((1 + (wacc / 100)) ** (percentage_remaining + 3.5)))

## Exit Multiple Method EV/EBITDA, Mature & Stable Industries
dcf_terminal = round((ebitda_num * evebitda_num) / ((1 + (wacc / 100)) ** (percentage_remaining + 3.5)))

ev = dcf_1 + dcf_2 + dcf_3 + dcf_4 + dcf_5 + dcf_terminal

print(f"Latest {current_year - 1} Free Cash Flow:          {fcf_ttm:,}")
print(f"Predicted {current_year} Discounted Cash Flow: {dcf_1:,}")
print(f"Predicted {current_year + 1} Discounted Cash Flow: {dcf_2:,}")
print(f"Predicted {current_year + 2} Discounted Cash Flow: {dcf_3:,}")
print(f"Predicted {current_year + 3} Discounted Cash Flow: {dcf_4:,}")
print(f"Predicted {current_year + 4} Discounted Cash Flow: {dcf_5:,}")
print(f"Predicted Terminal value of Discounted Cash Flow:  {dcf_terminal:,}")
print("==================================================================")
print(f"Net Present Value of {ticker}: {ev:,}")

Latest 2023 Free Cash Flow:          3,785,000,000
Predicted 2024 Discounted Cash Flow: 4,058,718,056
Predicted 2025 Discounted Cash Flow: 4,086,694,651
Predicted 2026 Discounted Cash Flow: 4,073,357,640
Predicted 2027 Discounted Cash Flow: 4,155,792,076
Predicted 2028 Discounted Cash Flow: 4,116,188,028
Predicted Terminal value of Discounted Cash Flow:  83,909,388,310
Net Present Value of SBUX: 104,400,138,761


In [49]:
options = Options()
options.add_argument('--headless')
options.add_argument('window-size = 800x600')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options = options)

ccs_url = fr"https://stockanalysis.com/stocks/{ticker}/financials/balance-sheet/"
driver.get(ccs_url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

ccs_td = soup.find('span', string = 'Cash & Cash Equivalents').find_parent('td').find_next('td').text
ccs_value = round(float(re.sub(',', '', ccs_td)) * 1e6)
print(f'Cash & Cash Equivalents: {ccs_value:,}')

driver.quit()

Cash & Cash Equivalents: 3,953,000,000


In [50]:
equity = ev - debt_value + ccs_value
target_price = round(equity / share_num, 2)

print(f"Target Price: {target_price}")
print(f"Current Price: {price}")

Target Price: 73.97
Current Price: 94.9


In [51]:
g = (price * share_num - dcf_terminal) / fcf_value
# c = terminal_growth_rate / (((wacc / 100) - (terminal_growth_rate - 1)) * ((1 + wacc / 100) ** 5))

x = symbols('x')
quintic_equation = x**5 + x**4 + x**3 + x**2 + x - g
solutions = solve(quintic_equation, x)
numerical_solutions = [sol.evalf() for sol in solutions]
t = float(numerical_solutions[0])
r = t * terminal_growth_rate - 1

print(f'Implied Growth Rate by current price: {round(r * 100, 4)} %')

Implied Growth Rate by current price: 10.0431 %
